In [1]:
import pandas as pd
import datetime as dt

In [2]:
path = '~/DataSet/RFM_data.csv'
df = pd.read_csv(path,low_memory = False)

In [3]:
df.head()

,InvoiceNo,CustomerCode,InvoiceDate,Amount
0,C0011810010001,19067290,2020-09-01,1716.00
1,C0011810010017,13233933,2020-09-01,1489.74
2,C0011810010020,99057968,2020-09-01,151.47
3,C0011810010021,80007276,2020-09-01,146.72
4,C0011810010024,13164076,2020-09-01,104.00


In [4]:
df.dtypes

InvoiceNo        object
CustomerCode     object
InvoiceDate      object
Amount          float64
dtype: object

In [5]:
# Просмотрим информацию о датасете
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 332730 entries, 0 to 332729
Data columns (total 4 columns):
 #   Column        Non-Null Count   Dtype  
---  ------        --------------   -----  
 0   InvoiceNo     332730 non-null  object 
 1   CustomerCode  332730 non-null  object 
 2   InvoiceDate   332730 non-null  object 
 3   Amount        332730 non-null  float64
dtypes: float64(1), object(3)
memory usage: 10.2+ MB


In [6]:
# Переводим столбцы в нужный тип данных
df['InvoiceDate'] = pd.to_datetime(df['InvoiceDate'])
df['InvoiceNo'] = df.InvoiceNo.apply(str)
df['CustomerCode'] = df.CustomerCode.apply(str)

In [7]:
# Определим минимальную и максимальную дату для подсчета Recency
df['InvoiceDate'].describe()

count                           332730
mean     2020-09-15 16:35:03.846361856
min                2020-09-01 00:00:00
25%                2020-09-08 00:00:00
50%                2020-09-16 00:00:00
75%                2020-09-24 00:00:00
max                2020-09-30 00:00:00
Name: InvoiceDate, dtype: object

In [8]:
# Максимальная дата
last_date = df['InvoiceDate'].max()

In [9]:
# Создаем новую таблицу с подсчетом RFM по каждому пользователю
RFMtable = df.groupby('CustomerCode',as_index = False).agg({'InvoiceDate':lambda x: (last_date - x.max()).days,
                                        'InvoiceNo': lambda x: len(x),
                                        'Amount': lambda x: x.sum()})
                                                        

In [10]:
# Переименовываем столбцы для каждого показателя
RFMtable = RFMtable.rename(columns = {'InvoiceDate':'Recency','InvoiceNo':'Frequency','Amount':'Monetary'})

In [11]:
RFMtable['Recency'] = RFMtable['Recency'].astype(int)

In [12]:
RFMtable

,CustomerCode,Recency,Frequency,Monetary
0,02213019,19,1,1609.20
1,02213042,22,3,9685.48
2,02213071,29,1,415.00
3,02213088,23,1,305.00
4,02213092,25,1,1412.88
...,...,...,...,...
123728,99099927,10,1,961.10
123729,99099936,0,1,1521.78
123730,99099959,8,2,1444.56
123731,99099963,19,1,3018.91


In [13]:
RFMtable.dtypes

CustomerCode     object
Recency           int64
Frequency         int64
Monetary        float64
dtype: object

#### Дальше определяем квантили для распределения пользователей по сегментам.Обычно это 25, 50, 75 квантили, но иногда следует использовать собсетвенные квантили. Все зависит от распределения ваших данных. В нашем случае будем использовать базовые квантили.

In [14]:
quantiles = RFMtable[['Recency','Frequency','Monetary']].quantile(q=[0.25,0.5,0.75])

In [15]:
quantiles

,Recency,Frequency,Monetary
0.25,2.0,1.0,765.00
0.50,8.0,2.0,1834.48
0.75,16.0,3.0,4008.84


#### Если в сегменте Recency пользователь находится в 25 квантиле, то ему присваивается 1 группа, а если в 75 квантиле или выше, то 4 группа. В случае с Frequency и Monetary все наоборот, потому что пользователь, который совершил больше покупок и на большую сумму находится в 1 группе (75 квантиль).

In [16]:
# Создаем функцию для распредления пользователей по группам
def RClass(value,parameter_name,quantiles_table):
    if value <= quantiles_table[parameter_name][0.25]:
        return 1
    elif value <= quantiles_table[parameter_name][0.50]:
        return 2
    elif value <= quantiles_table[parameter_name][0.75]:
        return 3 
    else:
        return 4

def FMClass(value,parameter_name,quantiles_table):
    if value <= quantiles_table[parameter_name][0.25]:
        return 4
    elif value <= quantiles_table[parameter_name][0.50]:
        return 3
    elif value <= quantiles_table[parameter_name][0.75]:
        return 2 
    else:
        return 1

In [17]:
# Применяем функции к новым колонкам в таблице
RFMtable['R _Quartile'] = RFMtable['Recency'].apply(RClass, args=('Recency', quantiles))
RFMtable['F_Quartile'] = RFMtable[ 'Frequency'].apply(FMClass, args=( 'Frequency', quantiles))
RFMtable['M_Quartile'] = RFMtable[ 'Monetary'].apply(FMClass, args =( 'Monetary',quantiles))

In [18]:
# Объеденим все результаты для получения класса пользователя
RFMtable['RFM_class'] = RFMtable['R _Quartile'].map(str) + RFMtable['F_Quartile'].map(str) + RFMtable['M_Quartile'].map(str)

In [19]:
RFMtable

,CustomerCode,Recency,Frequency,Monetary,R _Quartile,F_Quartile,M_Quartile,RFM_class
0,02213019,19,1,1609.20,4,4,3,443
1,02213042,22,3,9685.48,4,2,1,421
2,02213071,29,1,415.00,4,4,4,444
3,02213088,23,1,305.00,4,4,4,444
4,02213092,25,1,1412.88,4,4,3,443
...,...,...,...,...,...,...,...,...
123728,99099927,10,1,961.10,3,4,3,343
123729,99099936,0,1,1521.78,1,4,3,143
123730,99099959,8,2,1444.56,2,3,3,233
123731,99099963,19,1,3018.91,4,4,2,442


In [20]:
# Посмотрим на распределение классов в таблице
RFMtable['RFM_class'].value_counts().sort_values(ascending = False)

RFM_class
444    10624
111     9705
443     6729
344     6593
211     5847
       ...  
424       63
114       60
214       60
314       33
414        2
Name: count, Length: 64, dtype: int64

### Считается, что пользователь класса 111 являются лучшими, потому что часто покупает и на большую сумму, в то время как пользователя класса 444 являются худшими, потому что давно покупали и на маленькую сумму.